### 1. Amostra aleatória simples <- Recomendada em caso de apenas placas

In [ ]:
import pandas as pd
import numpy as np

# Exemplo: base de 1.4M linhas com estados
np.random.seed(42)
dados = pd.DataFrame({
    "estado": np.random.choice(
        ["SP", "RJ", "MG", "BA", "RS"], 1_400_000, p=[0.5, 0.2, 0.15, 0.1, 0.05]
    )
})

# Amostra simples de 10 mil registros
amostra = dados.sample(n=10_000, random_state=42)

print(amostra["estado"].value_counts(normalize=True))


Como funciona:
Cada elemento da população tem a mesma probabilidade de ser selecionado.
O algoritmo geralmente usa um gerador de números pseudoaleatórios para escolher índices da base.  

No pandas, o método ***DataFrame.sample()*** implementa isso — ele embaralha as linhas internamente e retorna um subconjunto.  


Vantagens:
- Simples de implementar.
- Produz uma amostra não tendenciosa se a base for homogênea.

Limitações:
- Pode não representar bem subgrupos menores (ex: estados com menos dados podem ficar sub-representados).
- Requer a base completa e acesso a todas as observações.

Usos típicos:
- Pesquisas gerais, testes de desempenho, validação rápida de dados.

---
### 2. Amostra balanceada (estratificada) <- Recomendada em caso de colunas extras

In [ ]:
from sklearn.model_selection import train_test_split

# Estratificando pela coluna "estado"
amostra_balanceada, _ = train_test_split(
    dados, 
    train_size=10_000, 
    stratify=dados["estado"], 
    random_state=42
)

print(amostra_balanceada["estado"].value_counts())


Como funciona:
A população é dividida em estratos (grupos homogêneos) — por exemplo, “Estado”, “Gênero”, “Faixa Etária”.
Depois, amostras são retiradas de cada estrato proporcionalmente ao seu tamanho na população.
O resultado mantém as mesmas proporções de cada grupo da base original.

Em Python, ***train_test_split(..., stratify=coluna)*** faz exatamente isso:
internamente, ele segmenta a base por classe e sorteia amostras proporcionais dentro de cada estrato antes de combiná-las novamente.

Vantagens:
- Garante representatividade de todos os grupos.
- Reduz a variância da estimativa global.

Limitações:
- Exige conhecer a variável de estratificação.
- Pode ser mais complexo se houver muitos grupos pequenos.

Usos típicos:
- Pesquisas populacionais, modelagem estatística, machine learning supervisionado (para preservar proporções de classes no treino e teste).

---
### 3. “Balanceada de verdade” (proporções iguais entre classes)

In [ ]:
# Número igual de amostras por estado
n = 2000
amostra_igual = (
    dados.groupby("estado")
    .apply(lambda x: x.sample(n=n, random_state=42))
    .reset_index(drop=True)
)

print(amostra_igual["estado"].value_counts())


Como funciona:
Diferente da estratificada, aqui forçamos cada classe ou grupo a ter o mesmo número de amostras, independentemente da proporção original.
É comum em problemas de classificação desbalanceados, como quando temos 90% de um tipo e 10% de outro.

O processo técnico:
- Divide-se a base em grupos (groupby).
- Dentro de cada grupo, seleciona-se o mesmo número de amostras 𝑛
- Une-se os subconjuntos, formando uma amostra balanceada.

Em Python, pode ser feito com:

In [ ]:
dados.groupby('classe').apply(lambda x: x.sample(n=minimo_comum))

Vantagens:
- Impede que classes majoritárias dominem o modelo.
- Ideal para treinar algoritmos de machine learning.

Limitações:
- Pode reduzir o tamanho total da amostra.
- Perde-se a proporcionalidade real dos dados.

Pode introduzir bias artificial se usado fora de contexto (ex: em inferência populacional).

Usos típicos:
- Modelos preditivos, classificação com classes desbalanceadas, detecção de fraudes